# Tutorial: Approximate Global Text Explaination (AGTE)
Explaining an IMDb Movies Text Classification Tensorflow Model globally for a dataset with Approximate Global Text Explaination via linear models.

In [ ]:
import re
import string
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import pandas as pd

from tensorflow.python.keras import Sequential
from tensorflow.python.keras.layers import Embedding, Dropout, GlobalAveragePooling1D, Dense
from tensorflow.python.keras.losses import BinaryCrossentropy
from tensorflow.python.keras.metrics import BinaryAccuracy
from ate.base import AGTE, ATE_Options

In [ ]:
MAX_FEATURES = 10000
EMBEDDING_DIM = 16
SEQUENCE_LENGTH = 250
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

In [ ]:
raw_train_ds = tfds.load('imdb_reviews', split='train').batch(BATCH_SIZE)
raw_test_ds = tfds.load('imdb_reviews', split='test').batch(BATCH_SIZE)

In [ ]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, '<br />', ' ')
    return tf.strings.regex_replace(stripped_html, '[%s]' % re.escape(string.punctuation), '')

In [ ]:
vectorize_layer = tf.keras.layers.TextVectorization(
    standardize=custom_standardization,
    max_tokens=MAX_FEATURES,
    output_mode='int',
    output_sequence_length=SEQUENCE_LENGTH)

In [ ]:
train_text = raw_train_ds.map(lambda x: x['text'])
vectorize_layer.adapt(train_text)

In [ ]:
def vectorize_text(text, label):
    text = tf.expand_dims(text, -1)
    return vectorize_layer(text), label

In [ ]:
train_ds = raw_train_ds.map(lambda x: vectorize_text(x['text'], x['label']))
test_ds = raw_test_ds.map(lambda x: vectorize_text(x['text'], x['label']))

In [ ]:
train_ds = train_ds.cache().prefetch(buffer_size=AUTOTUNE)
test_ds = test_ds.cache().prefetch(buffer_size=AUTOTUNE)

In [ ]:
model = Sequential([
    Embedding(MAX_FEATURES + 1, EMBEDDING_DIM),
    Dropout(0.2),
    GlobalAveragePooling1D(),
    Dropout(0.2),
    Dense(1)
])

In [ ]:
model.compile(loss=BinaryCrossentropy(from_logits=True),
              optimizer='adam',
              metrics=BinaryAccuracy(threshold=0.0))

In [ ]:
history = model.fit(train_ds, epochs=5)

In [ ]:
loss, accuracy = model.evaluate(test_ds)

print("Loss: ", loss)
print("Accuracy: ", accuracy)

In [ ]:
ate = AGTE(
    lambda x: np.array(x.lower().split()),
    lambda x: x, #INFO: Only needed for effect transformation.
    lambda x: model.predict([vectorize_layer(' '.join(e)).numpy().tolist() if len(e) > 0 else [0]*SEQUENCE_LENGTH for e in x['text'].tolist()])
)
options = ATE_Options(['text'], 2, 100, 1, 2)

In [ ]:
test_df = tfds.as_dataframe(raw_test_ds).head(10)
test_df = pd.DataFrame([i['text'][0].decode('UTF-8') for _, i in test_df.iterrows()], columns=['text'])
effects = ate.transform_effects(ate.explain(test_df, options))

In [ ]:
pd.DataFrame(effects)